In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf

### According to the formula of learning rate

In [4]:
# def optimizer_exp_decay():
#     global_step = tf.contrib.framework.get_or_create_global_step()
#     learning_rate = tf.train.exponential_decay(
#         learning_rate=0.1, global_step=global_step,
#         decay_steps=100, decay_rate=0.001)
#     return tf.train.AdagradOptimizer(learning_rate=learning_rate)
# sess = tf.Session()
# with sess.as_default():
#     obj = optimizer_exp_decay()
#     sess.run(tf.global_variables_initializer())


## TESTING LEARNING_RATE DECAY WITH A RANDOM EXAMPLE:

In [10]:
# Declare the global Variables :
from six.moves import xrange
from tensorflow.python.framework import ops

numFeatures = 3
trainSize=36
batchSize = 4
numEpochs = 10
numLabels = 2
numOut = 2 
starter_learning_rate = 0.01
momentum = 0.9

In [14]:
# Build Graph
ops.reset_default_graph()

global_step = tf.Variable(0, dtype=tf.float32)
learning_rate = tf.train.exponential_decay(starter_learning_rate, 
                                           global_step*batchSize,    # Used for decay computation
                                           trainSize,              # Decay steps
                                           0.95,           # Decay rate
                                           staircase=True) # Will decay the learning rate in discrete interval

# Creating a 1 layer Neural Network, to test the learning Rate
# define some Random weights



w1 = tf.Variable(tf.random_normal([numFeatures,numOut],seed=984))
b1 = tf.Variable(tf.random_normal([numOut],seed=984))
# print (w1.get_shape().as_list(), b1.get_shape().as_list())
# def network():
xIN = tf.placeholder(tf.float32, [None, numFeatures])
yIN = tf.placeholder(tf.float32, [None, numLabels])
linOUT = tf.matmul(xIN, w1) + b1
nonlinOUT = tf.nn.relu(linOUT)

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=nonlinOUT, labels=yIN))
# optimizer = (
#     tf.train.GradientDescentOptimizer(learning_rate)
#     .minimize(loss, global_step=global_step)
# )

optimizer = (
    tf.train.RMSPropOptimizer(learning_rate=learning_rate,
                              momentum=momentum)
    .minimize(loss, global_step=global_step)
)  



#### Some details:

* numEpochs : 10
* numSteps = int(numEpochs x trainSize)/batchSize
* numSteps = int(10 x 36)/4 = 90

* trainSize = 36
* batchSize = 4
* globalStep = global_step x batchSize
   * Global step increases by 1 every iteration. So when globalStep = global_step then we have simply [1,2,3] after 3 iteration. But now we have gloabalStep = global_step x batchSize which is [1x4,2x4,3x4] whith batchSize =4. So GlobalStep at 9th iteration = 4x9 = 36. Hold this in mind...
* decaySteps = trainSize
   * decay Steps basiclly says at which iteration should the learning rate decay. Here we mention that decay steps should be equal to trainSize which is 36. From above we saw that the GlobalSteps = 36 at 9th iteration. So we concur that after every 9th iteration the learning rate will decay.

If you dont believe on the above justification then try changing the decaySteps to 36. You will for sure see that the learning_rate will decay at step 37.



In [15]:
# Cteate a fake data set that matches the dimensions of MNIST examples
# Source : https://github.com/tensorflow/models/blob/master/tutorials/image/mnist/convolutional.py#L243

def fake_data(numDataPoints, numFeatures):
    """Generate a fake dataset that matches the dimensions of MNIST."""
    data = np.ndarray(
        shape=(numDataPoints, numFeatures),
        dtype=np.float32)
    labels = np.zeros(shape=(numDataPoints,), dtype=np.int64)
    for image in xrange(numDataPoints):
        label = image % 2
        data[image, 0] = label - 0.5
        labels[image] = label
    return data, labels


trainData, trainLabels = fake_data(trainSize, numFeatures)
numSteps = int(numEpochs*trainSize)//batchSize
print (numSteps)
sess = tf.Session()
with sess.as_default():
    sess.run(tf.global_variables_initializer())
    for step in xrange(int(numEpochs*trainSize)//batchSize):
        print ('##########################################')
        offset = (step * batchSize) % (trainSize - batchSize)
        print ('Step and offset : ', step, offset)
        batchData = trainData[offset:(offset + batchSize), ...]
        batchLabels = trainLabels[offset:(offset + batchSize)]
        batchLabels = np.eye(numLabels)[batchLabels]
        print(batchData.shape, batchLabels.shape)
        print ('')
        feedDict = {xIN:batchData, yIN:batchLabels}
        x_in, y_in, lin_out, nlin_out, ls, _, l_rate, g_step = sess.run([xIN, yIN, linOUT, 
                                                                             nonlinOUT, loss, optimizer, 
                                                                             learning_rate, global_step], feed_dict=feedDict)


        print ('Input Data: ', x_in)
        print ('Labels :', y_in )
        print ('linOUT : ', lin_out)
        print ('nonlinOUT : ', nlin_out)
        print ('loss : ', ls)
        print ('learning rate : ', l_rate)
        print ('global_step : ', g_step)
        print ('')

    

90
##########################################
Step and offset :  0 0
(4, 3) (4, 2)

Input Data:  [[ -5.00000000e-01  -1.58456325e+29   0.00000000e+00]
 [  5.00000000e-01   9.22803782e-37   1.40129846e-45]
 [ -5.00000000e-01   0.00000000e+00  -5.80708913e+19]
 [  5.00000000e-01   1.48181204e-27   1.40129846e-45]]
Labels : [[ 1.  0.]
 [ 0.  1.]
 [ 1.  0.]
 [ 0.  1.]]
linOUT :  [[ -2.56222477e+29   2.77942057e+28]
 [  4.91726026e-02   2.81464243e+00]
 [  1.23999843e+19  -1.73023108e+19]
 [  4.91726026e-02   2.81464243e+00]]
nonlinOUT :  [[  0.00000000e+00   2.77942057e+28]
 [  4.91726026e-02   2.81464243e+00]
 [  1.23999843e+19   0.00000000e+00]
 [  4.91726026e-02   2.81464243e+00]]
loss :  6.94855e+27
learning rate :  0.01
global_step :  1.0

##########################################
Step and offset :  1 4
(4, 3) (4, 2)

Input Data:  [[ -5.00000000e-01   1.40129846e-45   7.76133083e+11]
 [  5.00000000e-01   9.51156217e-26   1.40129846e-45]
 [ -5.00000000e-01   1.40129846e-45   2.9754728

## Tensor Board (Graph)

In [3]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf

tensorboard --logdir="/Users/sam/All-Program/App-DataSet/CIFAR-10/TensorBoard/events.out.tfevents.1504580050.Sardhendus-MacBook-Pro.local"

outputFile = "/Users/sam/All-Program/App-DataSet/CIFAR-10/TensorBoard/"
a = tf.add(1,2,)
b = tf.multiply(a,3)
c = tf.add(4,5,)
d = tf.multiply(c,6,)
e = tf.multiply(4,5,)
f = tf.div(c,6,)
g = tf.add(b,d)
h = tf.multiply(g,f)

with tf.Session() as sess:
    writer = tf.summary.FileWriter(outputFile, sess.graph)
    print (sess.run(h))
    writer.close()

63
